In [ ]:
import pandas as pd
import numpy as np
import os

# Load the Excel file
excel_file = pd.ExcelFile('Template v2.xlsx')

# Get the list of sheet names
sheet_names = excel_file.sheet_names

# Print the list of sheet names
print(f'Available sheets in the file are- {sheet_names}')

In [524]:
# Load the sheets into dataframes
Mapping_df = pd.read_excel(excel_file, sheet_name="Mapping")
BaseCapacity_df = pd.read_excel(excel_file, sheet_name="Base Capacity")
Support_Capacity_df = pd.read_excel(excel_file, sheet_name="Support Capacity")
Support_df = pd.read_excel(excel_file, sheet_name="Support")
Bi_Inputs_df = pd.read_excel(excel_file, sheet_name="BI Inputs")

In [525]:
numerical_col = ['Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']

In [527]:
#Mapping Sheet Data Workings

# Select only the "OU" and "Facility Name" columns
OU_Facility_df = Mapping_df[['OU', 'Facility Name']]
# Remove duplicates from OU_Facility_df
OU_Facility_df = OU_Facility_df.drop_duplicates(subset='OU')

# Select only the "OU", and "City" columns
OU_City_df = Mapping_df[['OU', 'City']]
# Remove duplicates from OU_City_df
OU_City_df = OU_City_df.drop_duplicates(subset='OU')

# Select only the "OU", and "BI" columns this is only used for BI inputs BI Geo validation
OU_BIGeo_df = Mapping_df[['OU', 'BI Geo']]
# Remove duplicates from OU_BIGeo_df
OU_BIGeo_df = OU_BIGeo_df.drop_duplicates(subset='OU')

# Select only the "BI Geo", and "Geo" columns
BIGeo_Geo_df = Mapping_df[['BI Geo', 'Geo']]
# Remove duplicates from BIGeo_Geo_df This is mandatory to remove duplicate entries
BIGeo_Geo_df = BIGeo_Geo_df.drop_duplicates(subset='BI Geo')

# Select only the "BI Geo", and "Geo Type" columns
BIGeo_GeoType_df = Mapping_df[['BI Geo', 'Geo Type']]
# Remove duplicates from BIGeo_GeoType_df This is mandatory to remove duplicate entries
BIGeo_GeoType_df = BIGeo_GeoType_df.drop_duplicates(subset='BI Geo')


In [528]:
#BI Inputs Sheet Data Working
# Rename specific columns
Bi_Inputs_df = Bi_Inputs_df.rename(columns={
    "LE[Scenario]": "FCST",
    "LE[Client]": "Customer",
    "LE[Horizontal]": "Horizontal",
    "LE[Stage]": "Stage",
    "LE[Vertical]": "Vertical",
    "LE[SD_Geo]": "BI Geo",
    "LE[OU_DESCR]": "OU",
    "LE[Project_DESCR]": "Program Name",
    "LE[Account]": "Account",
    "LE[Year]": "FY",
    "[SumJul]": "Jul",
    "[SumAug]": "Aug",
    "[SumSep]": "Sep",
    "[SumOct]": "Oct",
    "[SumNov]": "Nov",
    "[SumDec]": "Dec",
    "[SumJan]": "Jan",
    "[SumFeb]": "Feb",
    "[SumMar]": "Mar",
    "[SumApr]": "Apr",
    "[SumMay]": "May",
    "[SumJun]": "Jun"
})


columns_to_keep = [
    "FCST", "Customer", "Horizontal", "Stage", "Vertical", 
    "BI Geo", "OU", "Program Name", "FY", "Account"]+numerical_col
    
# Select only the specified columns
Bi_Inputs_df = Bi_Inputs_df[columns_to_keep]

#Fillter Account to select only Seats for Allocation_Adj_Store
Bi_Inputs_df = Bi_Inputs_df[Bi_Inputs_df['Account'] == "Seats for Allocation_Adj_Store"]


# Define a function to determine the 'Seat Type' based on the 'Stage' value
def determine_seat_type(stage):
    if stage == 'Existing':
        return 'Production'
    elif stage == 'Stage 5':
        return 'BD Stage 5'
    elif stage in ['Stage 3', 'Stage 4']:
        return 'BD Stage 3 & 4'
    else:
        return 'BD Stage 2 & Below'
    
# Apply the function to create the new column 'Seat Type'
Bi_Inputs_df['Seat Type'] = Bi_Inputs_df['Stage'].apply(determine_seat_type)

# Keep only the first 4 characters of the 'OU' column
Bi_Inputs_df['OU'] = Bi_Inputs_df['OU'].str.slice(0, 4)

# Remove the first 2 characters from the 'Customer' column
Bi_Inputs_df['Customer'] = Bi_Inputs_df['Customer'].str.slice(2)

# Remove the last 2 characters from the 'Vertical' column
Bi_Inputs_df['Vertical'] = Bi_Inputs_df['Vertical'].str.slice(0, -2)

# Remove the last 2 characters from the 'Horizontal' column
Bi_Inputs_df['Horizontal'] = Bi_Inputs_df['Horizontal'].str.slice(0, -2)

# Create a new column 'Job Code' based on the provided conditions
Bi_Inputs_df['Job Code'] = Bi_Inputs_df['Program Name'].apply(lambda x: x[:5] if x[:1].isdigit() else x[:18])

# Merge the dataframes on the 'OU' column
Bi_Inputs_df = pd.merge(Bi_Inputs_df, OU_Facility_df, on='OU', how='left')
Bi_Inputs_df = pd.merge(Bi_Inputs_df, OU_City_df, on='OU', how='left')
Bi_Inputs_df = pd.merge(Bi_Inputs_df, OU_BIGeo_df, on='OU', how='left')


# Replace 'BI Geo_x' with 'BI Geo_y' where they are not equal
Bi_Inputs_df['BI Geo_x'] = np.where(Bi_Inputs_df['BI Geo_x'] != Bi_Inputs_df['BI Geo_y'], Bi_Inputs_df['BI Geo_y'], Bi_Inputs_df['BI Geo_x'])

# Drop the 'BI Geo_y' column
Bi_Inputs_df = Bi_Inputs_df.drop(columns=['BI Geo_y'])

# Rename 'BI Geo_x' to 'BI Geo'
Bi_Inputs_df = Bi_Inputs_df.rename(columns={'BI Geo_x': 'BI Geo'})

# replace NaN values with zero. This is optional if incase there is no value provided.
Bi_Inputs_df[numerical_col] = Bi_Inputs_df[numerical_col].fillna(0)

# Convert month column data types to integer using numerical_col
Bi_Inputs_df[numerical_col] = Bi_Inputs_df[numerical_col].astype(int)


In [529]:
# New Working to get all row items

# Create combined index of all unique rows based on OU and BI Geo
all_rows = pd.concat([BaseCapacity_df[["OU","BI Geo"]],Support_Capacity_df[["OU","BI Geo"]],Support_df[["OU","BI Geo"]],Bi_Inputs_df[["OU","BI Geo"]]]).drop_duplicates()

# reindex Based Capacity to include all unique rows
BaseCapacity_df = BaseCapacity_df.set_index(["OU","BI Geo"]).reindex(all_rows.set_index(["OU","BI Geo"]).index,fill_value=0).reset_index()
Support_Capacity_df = Support_Capacity_df.set_index(["OU","BI Geo"]).reindex(all_rows.set_index(["OU","BI Geo"]).index,fill_value=0).reset_index()

# Add other columns data
# Identify the most common non-zero value in the "FY" column
most_common_value = BaseCapacity_df.loc[BaseCapacity_df["FY"] != 0, "FY"].mode()[0]
most_common_value = Support_Capacity_df.loc[Support_Capacity_df["FY"] != 0, "FY"].mode()[0]

# Replace 0 with the most common non-zero value using a lambda function
BaseCapacity_df["FY"] = BaseCapacity_df["FY"].apply(lambda x: most_common_value if x == 0 else x)
Support_Capacity_df["FY"] = Support_Capacity_df["FY"].apply(lambda x: most_common_value if x == 0 else x)

# You can use the inplace parameter to modify the DataFrame directly
BaseCapacity_df.drop(["Facility Name"], axis=1, inplace=True)
Support_Capacity_df.drop(["Facility Name"], axis=1, inplace=True)

# Add Facility Name Again for all rows based on OU
BaseCapacity_df = pd.merge(BaseCapacity_df, OU_Facility_df, on='OU', how='left')
Support_Capacity_df = pd.merge(Support_Capacity_df, OU_Facility_df, on='OU', how='left')

BaseCapacity_df["Seat Type"] = "Base Capacity"
Support_Capacity_df["Seat Type"] = "Support Capacity"

In [ ]:
# Extract the value from the first row of the "FCST" column
Scenario = Bi_Inputs_df.loc[0, "FCST"]

Scenario

In [532]:
Existing_Total_df = Bi_Inputs_df
BD_3andAbove_Total_df = Bi_Inputs_df
BD_2andBelow_Total_df = Bi_Inputs_df

In [533]:
StageLevelExisting = ['Existing']
StageLevel3to5 = ['Stage 3','Stage 4','Stage 5']
StageLevel2toBelow = ['Stage 1', 'Stage 2', 'Stage_Other']

In [534]:
# Filter the DataFrame to include only rows based on Stage
Existing_Total_df = Existing_Total_df[Existing_Total_df['Stage'].isin(StageLevelExisting)]
BD_3andAbove_Total_df = BD_3andAbove_Total_df[BD_3andAbove_Total_df['Stage'].isin(StageLevel3to5)]
BD_2andBelow_Total_df = BD_2andBelow_Total_df[BD_2andBelow_Total_df['Stage'].isin(StageLevel2toBelow)]

In [535]:
# Grouping the data to have single row item data for all 
Existing_Total_df = Existing_Total_df.groupby(["FY","FCST",'BI Geo',"Facility Name" ,'OU',"City"])[numerical_col].sum().reset_index().fillna(0)
BD_3andAbove_Total_df = BD_3andAbove_Total_df.groupby(["FY","FCST",'BI Geo',"Facility Name" ,'OU',"City"])[numerical_col].sum().reset_index().fillna(0)
BD_2andBelow_Total_df = BD_2andBelow_Total_df.groupby(["FY","FCST",'BI Geo',"Facility Name" ,'OU', "City"])[numerical_col].sum().reset_index().fillna(0)



In [536]:
#Base Capacity Sheet Data Working

# Grouping to sum up Base capacity if incase multiple entries or do manual check to remove duplicates from raw data
BaseCapacity_df = BaseCapacity_df.groupby(["FY",'OU','BI Geo',"Facility Name" ,"Seat Type"])[numerical_col].sum().reset_index().fillna(0)

# Merge the dataframes on the 'OU' column
BaseCapacity_df = pd.merge(BaseCapacity_df, OU_City_df, on='OU', how='left')

# Add FCST column
BaseCapacity_df['FCST']= Scenario

In [538]:
#Support Capacity Sheet Data Working

# Grouping to sum up Support capacity if incase multiple entries or do manual check to remove duplicates from raw data
Support_Capacity_df = Support_Capacity_df.groupby(["FY",'OU','BI Geo',"Facility Name" ,"Seat Type"])[numerical_col].sum().reset_index().fillna(0)

# Merge the dataframes on the 'OU' column
Support_Capacity_df = pd.merge(Support_Capacity_df, OU_City_df, on='OU', how='left')
# replace NaN values with zero. This is optional if incase there is no value provided.
Support_Capacity_df[numerical_col] = Support_Capacity_df[numerical_col].fillna(0)

# Add FCST column
Support_Capacity_df['FCST']= Scenario


In [539]:
#Support Sheet Data Working
#Fillter Sum column to remove rows if its value is 0
Support_df = Support_df[Support_df['Sum'] != 0]
# Merge the dataframes on the 'OU' column
Support_df = pd.merge(Support_df, OU_City_df, on='OU', how='left')
# replace NaN values with zero. This is optional if incase there is no value provided.
Support_df[numerical_col] = Support_df[numerical_col].fillna(0)

# Add FCST column
Support_df['FCST']= Scenario


In [540]:
# New Working for calculations
# Set index for each DataFrame without inplace=True
BaseCapacity_df = BaseCapacity_df.set_index(["OU", "Facility Name", "FY", "BI Geo", "City"])
Support_Capacity_df = Support_Capacity_df.set_index(["OU", "Facility Name", "FY", "BI Geo", "City"])

# Align the indexes and fill missing rows/values with 0
all_indexes = BaseCapacity_df.index.union(Support_Capacity_df.index).union(
    Existing_Total_df.set_index(["OU", "Facility Name", "FY", "BI Geo", "City"]).index
).union(
    BD_3andAbove_Total_df.set_index(["OU", "Facility Name", "FY", "BI Geo", "City"]).index
).union(
    BD_2andBelow_Total_df.set_index(["OU", "Facility Name", "FY", "BI Geo", "City"]).index
)

BaseCapacity_df = BaseCapacity_df.reindex(all_indexes, fill_value=0)
Support_Capacity_df = Support_Capacity_df.reindex(all_indexes, fill_value=0)
EX_df = Existing_Total_df.set_index(["OU", "Facility Name", "FY", "BI Geo", "City"]).reindex(all_indexes, fill_value=0)
BD_3andAbove_df = BD_3andAbove_Total_df.set_index(["OU", "Facility Name", "FY", "BI Geo", "City"]).reindex(all_indexes, fill_value=0)
BD_2andBelow_df = BD_2andBelow_Total_df.set_index(["OU", "Facility Name", "FY", "BI Geo", "City"]).reindex(all_indexes, fill_value=0)

# Perform the calculations with NaN values replaced by 0
FS_afterSD_df = BaseCapacity_df[numerical_col].fillna(0) - (Support_Capacity_df[numerical_col].fillna(0) + EX_df[numerical_col].fillna(0))
FS_afterBD3andAbove_df = BaseCapacity_df[numerical_col].fillna(0) - (Support_Capacity_df[numerical_col].fillna(0) + EX_df[numerical_col].fillna(0) + BD_3andAbove_df[numerical_col].fillna(0))
FS_afterAllBD_df = BaseCapacity_df[numerical_col].fillna(0) - (Support_Capacity_df[numerical_col].fillna(0) + EX_df[numerical_col].fillna(0) + BD_3andAbove_df[numerical_col].fillna(0) + BD_2andBelow_df[numerical_col].fillna(0))

# Reset index to bring back the index columns
FS_afterSD_df = FS_afterSD_df.reset_index()
FS_afterBD3andAbove_df = FS_afterBD3andAbove_df.reset_index()
FS_afterAllBD_df = FS_afterAllBD_df.reset_index()

# Add Seat Type column
FS_afterSD_df['Seat Type'] = "Free Seats After SD"
FS_afterBD3andAbove_df['Seat Type'] = "Free Seats After BD Stage 3 and Above"
FS_afterAllBD_df['Seat Type'] = "Free Seats After all BD"

# Add FCST column
FS_afterSD_df['FCST'] = Scenario
FS_afterBD3andAbove_df['FCST'] = Scenario
FS_afterAllBD_df['FCST'] = Scenario


In [541]:
# New working for Production Capacity
# Define the column headers
index_columns = ["FY", "BI Geo", "Facility Name", "OU",]

# Create an empty DataFrame with the specified headers
Pro_Capacity_df = pd.DataFrame(columns=index_columns)



In [542]:
BaseCapacity_df = BaseCapacity_df.reset_index()
Support_Capacity_df = Support_Capacity_df.reset_index()

In [543]:

# Set index for each DataFrame without inplace=True
BaseCapacity_df = BaseCapacity_df.set_index(index_columns)
Support_Capacity_df = Support_Capacity_df.set_index(index_columns)

# Create the combined index
all_indexes = BaseCapacity_df.index.union(Support_Capacity_df.index).union(
    Pro_Capacity_df.set_index(index_columns).index
)

BaseCapacity_df = BaseCapacity_df.reindex(all_indexes, fill_value=0)
Support_Capacity_df = Support_Capacity_df.reindex(all_indexes, fill_value=0)
Pro_Capacity_df = Pro_Capacity_df.set_index(index_columns).reindex(all_indexes, fill_value=0)

# Perform the calculations with NaN values replaced by 0
Pro_Capacity_df = BaseCapacity_df[numerical_col].fillna(0) - Support_Capacity_df[numerical_col].fillna(0)

# Reset index to bring back the index columns
Pro_Capacity_df = Pro_Capacity_df.reset_index()
# Reset index to bring back the index columns Trial only for now not sure if this is making any difference in final output
BaseCapacity_df = BaseCapacity_df.reset_index()
Support_Capacity_df = Support_Capacity_df.reset_index()
# Add Seat Type column
Pro_Capacity_df['Seat Type'] = "Production Capacity"

# Add FCST column
Pro_Capacity_df['FCST'] = Scenario

In [544]:
# Append all required dataframes to make final GCD

Final_df = pd.concat([BaseCapacity_df, Support_Capacity_df,Pro_Capacity_df, Support_df, Bi_Inputs_df, FS_afterSD_df,FS_afterBD3andAbove_df,FS_afterAllBD_df], ignore_index=True)

# Replace NaN values with 0 in the specified numerical columns
Final_df[numerical_col] = Final_df[numerical_col].fillna(0)

# Replace NaN values in all other columns with " - "
Final_df = Final_df.apply(lambda x: x.fillna(" - ") if x.name not in numerical_col else x)

# Convert month column data types to integer using numerical_col
Final_df[numerical_col] = Final_df[numerical_col].astype(int)

# Merge the dataframes on the 'BI Geo' column to get Geo
Final_df = pd.merge(Final_df, BIGeo_Geo_df, on='BI Geo', how='left')

# Merge the dataframes on the 'BI Geo' column to get Geo Type
Final_df = pd.merge(Final_df, BIGeo_GeoType_df, on='BI Geo', how='left')

# Create the new column 'SD Geo' by concatenating 'Geo Type' and 'Geo' columns
Final_df["SD Geo"] = Final_df["Geo Type"] + " " +Final_df["Geo"]
Final_df["SD Geo OU"]= Final_df["SD Geo"] + "_" + Final_df["OU"]




In [546]:
# Reordering the columns in Final GCD
Final_df = Final_df[['FCST', 'FY',  'BI Geo','Geo', "SD Geo","Geo Type","SD Geo OU",'OU', 'City', 'Facility Name', 'Vertical', 'Stage',
                     'Job Code', 'Customer', 'Seat Type','Account', 'Jul', 'Aug', 'Sep', 'Oct',
                     'Nov', 'Dec', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun']]

In [ ]:
#Write all required data to new excel file
# Try to export the merged dataframe to an Excel file
try:
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    with pd.ExcelWriter('PythonGCD.xlsx', engine='xlsxwriter') as writer:
        # Write each dataframe to a different worksheet.
        
        # Bi_Inputs_df.to_excel(writer, sheet_name='BI Inputs', index=False)
        Final_df.to_excel(writer, sheet_name='Final GCD', index=False)
    print("Data has been exported to 'PythonGCD.xlsx'")
except PermissionError:
    print("The file 'PythonGCD.xlsx' is already open. Please close the file and try again.")